In [183]:
import re
import pandas as pd
from g4f.client import Client
from g4f.Provider import RetryProvider, Phind, FreeChatgpt, Liaobots
import g4f.debug
g4f.debug.logging = True

In [184]:
df = pd.read_csv("data/Sustainability Courses Offered - 1 - Course Review.csv")
df.head()

,Course,Course URL,Subject,Last Offered,Course Title,Credits Offered,Assessment of Sustainability Focused or Inclusive,Offered to UG/G/Both or Other,Justification for classification (including quotes from syllabi/course descriptions where applicable),Description,Course Home Department,Last Catalog Year Reviewed,Unnamed: 12,"The courses were counted using catalog listings with language that is aligned with the Earth Charter, covering topics such as ecological integrity, social and economic justice, and democracy and nonviolence. Course descriptions that included terms like ""environment,"" ""sustainable/sustainability,"" and ""energy"" were counted as Sustainability-Focused courses, and the other courses that address a topic from the Earth Charter but are not centrally-focused on sustainability were considered Sustainability-Included courses.",Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,AAAS,https://courses.rice.edu/courses/courses/!SWKS...,200,Fall 2021,Knowing Blackness: Introduction To African And...,3,N,UG,Has a strong cultural emphasis of how the Afri...,An exploration of the origins and development ...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,Percentage of Sustainability Course Offerings
1,AAAS,https://courses.rice.edu/courses/courses/!SWKS...,300,Spring 2022,Writing Black Lives,3,Y - Inclusive,UG,This course focuses on Black Writers and their...,"In this course, students will be reading, refl...",African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,27.94%
2,AAAS,NaN,510,Spring 2022,Intro To Diasporic Studies,3,N,G,This course is for graduate students who are i...,This is the core course for the Certificate in...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAS,NaN,600,Fall 2021,African And African American Studies Colloquium,3,N,G,This course is for graduate students who are i...,Through readings and discussions the colloquiu...,African And African American Studies,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN
4,AFSC,https://courses.rice.edu/courses/courses/!SWKS...,101,Fall 2021,Foundation Of The Usaf I,2,N,UG,No indication of sustainability focus or inclu...,Overall roles and missions of the USAF; career...,Air Force Science,2021-2022,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
def create_client():
    provider=RetryProvider([Phind, FreeChatgpt, Liaobots], shuffle=False)
    client = Client(
        provider=provider
    )

    return client, provider

In [186]:
def generate_prompts(descriptions):
    prompts = []
    for i in descriptions:
        prompt = f"""A course description is provided below:\n\n{i}\n\nBased on this course description does this class promote sustainability? 
                    Please assess it with a 1 (sustainable) or 0 (not sustainable) and provide a one sentence explanation why.
                    The format should be 1 or 0: (Reasoning) (newline)"""
        prompts.append(prompt)

    return prompts

In [187]:
def run_model(client, provider, prompt):
    response = client.chat.completions.create(
        model="gemini",
        messages=[{"role": "user", "content": f"{prompt}"}],
        provider=provider
    )

    return response

In [188]:
def parse_response(response):
    eval = []
    reason = []
    print(response.choices[0].message.content)
    for i, j in [string.split(': ', 1) for string in response.choices[0].message.content.split('\n')]:
        eval.append(i)
        reason.append(j)
    
    return eval, reason

In [189]:
client, provider = create_client()

In [190]:
def run(client, provider, descriptions):
    count = 100
    evals, reasons = [], []
    while count < len(descriptions):
        eval, reason = parse_response(run_model(client, provider, descriptions[(count-100):count]))
        evals += eval
        reasons += reason
        count += 100
    eval, reason = parse_response(run_model(client, provider, descriptions[(count-100):]))
    evals += eval
    reasons += reason

    return evals, reasons

In [ ]:
evals, reasons = run(*create_client(), generate_prompts(df.Description))

In [191]:
response = run_model(*create_client(), generate_prompts(df.Description))

Using RetryProvider provider and gemini model
Using FreeChatgpt provider


KeyboardInterrupt: 

In [ ]:
eval, reason = parse_response(response)

1: Provides an exploration of the origins and development of African Studies and African American Studies. (Introduces students to methodologies and practices that have led to and that continue to lead to knowing Africa and African-descendent people with earnest regard for the complexity and subtlety that the subjects require.)
0: Students will be reading, reflecting, and dissecting short stories, novels, television scripts, and other works of fiction crafted by artists across the Black diaspora. (Does not promote sustainability.)
0: Provides an introduction to cross- and multi-disciplinary approaches to the histories, cultures and experiences of African and African Diasporic people, while also introducing students to the work of Rice faculty working in the field. (Does not promote sustainability.)
0: Through readings and discussions the colloquium highlights key issues related to African and African American studies for graduate students preparing to conduct research in the field. (Do

In [ ]:
print(eval)
print(reason)

['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0']
['Provides an exploration of the origins and development of African Studies and African American Studies. (Introduces students to methodologies and practices that have led to and that continue to lead to knowing Africa and African-descendent people with earnest regard for the complexity and subtlety that the subjects require.)', 'Students will be reading, reflecting, and dissecting short stories, novels, television scripts, and other works of fiction crafted by artists across the Black diaspora. (Does not promote sustainability.)', 'Provides an introduction to cross- and multi-disciplinary approaches to the histories, cultures and experiences of African and African Diasporic people, while also introducing students to the work of Rice faculty working in the field. (Does not promote sustainability.)', 'Through readings and discussions the colloquium highlights key issues re

In [ ]:
print(sum([int(i) for i in eval]))

4
